In [90]:
import pandas as pd
import pandas as pd
import geopandas as gpd
import geopy
from urllib.parse import quote
from sqlalchemy import create_engine
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
import os
from dotenv import load_dotenv

load_dotenv()

USER = os.getenv('USER')
PWD = os.getenv('PASSWORD')
LAKE = os.getenv('LAKE')
MART = os.getenv('MART')
PORT = os.getenv('PORT')
DB = os.getenv('DBNAME')

In [5]:
connection_string = f"postgresql+psycopg2://{USER}:%s@{LAKE}:{PORT}/{DB}"
pglake = create_engine(connection_string % quote(PWD))

df_raw = pd.read_sql("select * from taxi_trips", pglake)
# pglake.close()

In [53]:
df_raw = df_raw.loc[~((df_raw['pickupcentroidlatitude'] == "") | (df_raw["pickupcentroidlongitude"] == "") | (df_raw["dropoffcentroidlatitude"] == "") | (df_raw["dropoffcentroidlongitude"] == "" ))]
df_raw = df_raw.loc[~((df_raw['tripseconds'] == "") | (df_raw["tripmiles"] == ""))]

df_raw = df_raw.astype(
    {
    "pickupcentroidlatitude":"float64",
    "pickupcentroidlongitude":"float64",
    "dropoffcentroidlatitude":"float64",
    "dropoffcentroidlongitude":"float64"
     }
    )

df_raw[["pickupcentroidlatitude", "pickupcentroidlongitude", "dropoffcentroidlatitude", "dropoffcentroidlongitude"]] = round(df_raw[["pickupcentroidlatitude", "pickupcentroidlongitude", "dropoffcentroidlatitude", "dropoffcentroidlongitude"]],3)


In [59]:
coords1 = df_raw.drop_duplicates(['pickupcentroidlatitude','pickupcentroidlongitude'])[["pickupcentroidlongitude", "pickupcentroidlatitude"]]
coords2 = df_raw.drop_duplicates(['dropoffcentroidlatitude','dropoffcentroidlongitude'])[["dropoffcentroidlongitude", "dropoffcentroidlatitude"]]

coords = pd.concat(
    [
    coords1.rename(columns={"pickupcentroidlatitude":"latitude","pickupcentroidlongitude":"longitude"}).reset_index(drop=True),
     coords2.rename(columns={ "dropoffcentroidlatitude":"latitude", "dropoffcentroidlongitude":"longitude"}).reset_index(drop=True)
     ],
    axis=0
).drop_duplicates()

coords["coords"] = coords["latitude"].astype('str') + "," + coords["longitude"].astype('str')

locator = Nominatim(user_agent='myGeocoder', timeout=10)
rgeocode = RateLimiter(locator.reverse, min_delay_seconds=0.001)

get_zipcode = lambda row: rgeocode((row['latitude'], row['longitude'])).raw['address']['postcode']

coords['zip'] = coords.apply(get_zipcode, axis=1)

,longitude,latitude
0,-87.699,41.923
1,-87.914,41.980
2,-87.633,41.900
3,-87.620,41.857
4,-87.723,41.954
...,...,...
83,-87.670,41.900
87,-87.653,41.893
90,-87.634,41.922
91,-87.636,41.929


In [54]:
df_raw['pickupcoords'] = df_raw["pickupcentroidlatitude"].astype('str') + "," + df_raw["pickupcentroidlongitude"].astype('str')
df_raw['dropoffcoords'] = df_raw["dropoffcentroidlatitude"].astype('str') + "," + df_raw["dropoffcentroidlongitude"].astype('str')

df_raw = df_raw.merge(
    coords.drop_duplicates(),
    right_on= "coords",
    left_on="pickupcoords",
    how = 'left'
).rename(columns={"zip":"pickupzip"}).merge(
    coords.drop_duplicates(),
    right_on= "coords",
    left_on="dropoffcoords",
    how = 'left'

).rename(columns={"zip":"dropoffzip"})

In [62]:
df_raw['tripstarttimestamp'] = pd.to_datetime(df_raw['tripstarttimestamp'])
df_raw['tripendtimestamp'] = pd.to_datetime(df_raw['tripendtimestamp'])

df_raw = df_raw.astype(
    {'tripseconds':'float64'},
    {'tripmiles':'float64'}
)

In [49]:
df_raw.columns

Index(['tripid', 'taxiid', 'tripstarttimestamp', 'tripendtimestamp',
       'tripseconds', 'tripmiles', 'pickupcensustract', 'dropoffcensustract',
       'pickupcommunityarea', 'dropoffcommunityarea', 'fare', 'tips', 'tolls',
       'extras', 'triptotal', 'paymenttype', 'company',
       'pickupcentroidlatitude', 'pickupcentroidlongitude',
       'dropoffcentroidlatitude', 'dropoffcentroidlongitude', 'pickupcoords',
       'dropoffcoords', 'longitude_x', 'latitude_x', 'coords_x', 'pickupzip',
       'longitude_y', 'latitude_y', 'coords_y', 'dropoffzip'],
      dtype='object')

In [64]:
df = df_raw[["tripid","taxiid","tripstarttimestamp","tripendtimestamp","tripseconds","tripmiles","dropoffzip","pickupzip","pickupcoords","dropoffcoords"]]

In [68]:
df["tripdate"] = df["tripstarttimestamp"].dt.date
df["tripweekday"] = df["tripstarttimestamp"].dt.weekday

C:\Users\BrettMele\AppData\Local\Temp\ipykernel_18448\843996576.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["tripdate"] = df["tripstarttimestamp"].dt.date
C:\Users\BrettMele\AppData\Local\Temp\ipykernel_18448\843996576.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["tripweekday"] = df["tripstarttimestamp"].dt.weekday


In [75]:
df_agg1 = df.groupby(["tripdate","pickupzip"]).size().reset_index(name="count")
df_agg2 = df.groupby(["tripdate","dropoffzip"]).size().reset_index(name="count")

In [83]:
df_agg = pd.concat(
    [
    df_agg1.rename(columns={"pickupzip":"zip"}),
    df_agg2.rename(columns={"dropoffzip":"zip"})
    ]
).groupby(['tripdate','zip']).sum('count').reset_index()

In [84]:
df_agg

,tripdate,zip,count
0,2023-02-28,60601,22
1,2023-02-28,60603,45
2,2023-02-28,60604,155
3,2023-02-28,60605,22
4,2023-02-28,60607,102
...,...,...,...
62,2023-03-01,60651,2
63,2023-03-01,60653,1
64,2023-03-01,60657,1
65,2023-03-01,60660,1


In [91]:
# write to mart
connection_string = f"postgresql+psycopg2://{USER}:%s@{MART}:{PORT}/{DB}"
pgmart = create_engine(connection_string % quote(PWD))
conn = pgmart.connect()

df.to_sql("requirement_4_taxi_trips",schema="public", con = conn, if_exists="replace")

conn.close()